<a href="https://colab.research.google.com/github/siyasathaye/power-outage-analysis/blob/main/notebook/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Power Outage Analysis

**Name(s)**: Siya Sathaye & Risa Schloyer

**Website Link**: (your website link)

In [1]:
!wget https://raw.githubusercontent.com/dsc-courses/dsc80-2025-fa/main/projects/proj04/dsc80_utils.py

import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from dsc80_utils import * # Feel free to uncomment and use this.


--2025-12-10 00:27:05--  https://raw.githubusercontent.com/dsc-courses/dsc80-2025-fa/main/projects/proj04/dsc80_utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4939 (4.8K) [text/plain]
Saving to: ‘dsc80_utils.py.1’

dsc80_utils.py.1    100%[===================>]   4.82K  --.-KB/s    in 0s      

2025-12-10 00:27:05 (58.2 MB/s) - ‘dsc80_utils.py.1’ saved [4939/4939]



In [2]:
import plotly.io as pio
pio.renderers.default = "colab"

In [3]:
!pip install openpyxl

In [4]:
from google.colab import files
uploaded = files.upload()

Saving outage.xlsx to outage (1).xlsx


In [5]:
df = pd.read_excel("outage.xlsx")

## Step 1: Introduction

In [6]:
# TODO
# Question: How do severe weather events influence the duration and overall impact of major power outages across
# different regions in the United States?

## Step 2: Data Cleaning and Exploratory Data Analysis

In [7]:
# TODO
df = df.copy()
df = df.dropna(axis=1, how="all")
df = df.drop(columns=["OBS"], errors="ignore")

df["OUTAGE.START"] = pd.to_datetime(
    df["OUTAGE.START.DATE"].astype(str) + " " + df["OUTAGE.START.TIME"].astype(str),
    errors="coerce"
)
df["OUTAGE.RESTORATION"] = pd.to_datetime(
    df["OUTAGE.RESTORATION.DATE"].astype(str) + " " + df["OUTAGE.RESTORATION.TIME"].astype(str),
    errors="coerce"
)

df = df.drop(columns=[
    "OUTAGE.START.DATE",
    "OUTAGE.START.TIME",
    "OUTAGE.RESTORATION.DATE",
    "OUTAGE.RESTORATION.TIME"
])

num_cols = [
    "YEAR",
    "MONTH",
    "ANOMALY.LEVEL",
    "OUTAGE.DURATION",
    "DEMAND.LOSS.MW",
    "CUSTOMERS.AFFECTED"
]

for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df["YEAR"] = df["YEAR"].astype("Int64")
df["MONTH"] = df["MONTH"].astype("Int64")

df = df.rename(columns={"OUTAGE.DURATION": "OUTAGE.DURATION.MIN"})
#need to drop all irrelevant columns

In [8]:
fig = px.histogram(
    df,
    x="OUTAGE.DURATION.MIN",
    nbins=50,
    title="Distribution of Outage Duration (minutes)"
)
fig.update_layout(
    xaxis_title="Minutes",
    yaxis_title="Frequency"
)


fig.show()

In [9]:
counts = df["CAUSE.CATEGORY"].value_counts()

fig = px.bar(
    x=counts.index,
    y=counts.values,
    title="Counts of Major Outages by Cause Category"
)
fig.update_layout(
    xaxis_title="Cause category",
    yaxis_title="Number of outages"
)
fig.show()


In [10]:
df["IS_SEVERE"] = df["CAUSE.CATEGORY"] == 'severe weather'

In [11]:
fig = px.box(
    df,
    x="IS_SEVERE",
    y="OUTAGE.DURATION.MIN",
    title="Outage Duration for Severe Weather vs Other Causes",
    labels={"IS_SEVERE": "Is severe weather?", "OUTAGE.DURATION.MIN": "Duration (minutes)"}
)
fig.show()

In [12]:
severe_df = df[df["IS_SEVERE"]]

fig = px.box(
    severe_df,
    x="NERC.REGION",
    y="OUTAGE.DURATION.MIN",
    title="Outage Duration for Severe Weather Events by NERC Region",
    labels={"OUTAGE.DURATION.MIN": "Duration (minutes)", "NERC.REGION": "NERC Region"}
)
fig.show()

In [13]:
cause_agg = (
    df.groupby("CAUSE.CATEGORY")["OUTAGE.DURATION.MIN"]
      .agg(["mean", "median", "count"])
)
cause_agg

,mean,median,count
CAUSE.CATEGORY,,,
equipment failure,1816.91,221.0,55
fuel supply emergency,13484.03,3960.0,38
intentional attack,429.98,56.0,403
islanding,200.55,77.5,44
public appeal,1468.45,455.0,69
severe weather,3883.99,2460.0,744
system operability disruption,728.87,215.0,123


In [14]:
df.groupby("CLIMATE.CATEGORY")["OUTAGE.DURATION.MIN"].agg(['mean', 'median', 'count'])

,mean,median,count
CLIMATE.CATEGORY,,,
cold,2656.96,816.0,463
normal,2530.98,563.0,730
warm,2817.32,881.0,283


## Step 3: Assessment of Missingness

> Add blockquote



In [15]:
# Missingness indicator for CUSTOMERS.AFFECTED
df["CA_MISSING"] = df["CUSTOMERS.AFFECTED"].isna()
# Create categorical encoding (integer codes)
df["CAUSE_CODE"] = df["CAUSE.CATEGORY"].astype("category").cat.codes

In [16]:
obs_dep = (
    df.groupby("CA_MISSING")["CAUSE_CODE"].mean().loc[True]
    - df.groupby("CA_MISSING")["CAUSE_CODE"].mean().loc[False]
)
obs_dep


np.float64(-1.566310030973717)

In [17]:
N = 5000
null_stats_dep = []

for _ in range(N):
    shuffled = pd.Series(
        np.random.permutation(df["CA_MISSING"].values),
        index=df.index
    )

    means_perm = df.groupby(shuffled)["CAUSE_CODE"].mean()
    stat = means_perm.loc[True] - means_perm.loc[False]
    null_stats_dep.append(stat)

null_stats_dep = np.array(null_stats_dep)

p_value_dep = np.mean(np.abs(null_stats_dep) >= abs(obs_dep))
p_value_dep


np.float64(0.0)

In [18]:
import plotly.express as px

fig = px.histogram(
    null_stats_dep, nbins=40,
    title="Missingness of CUSTOMERS.AFFECTED vs CAUSE.CATEGORY"
)
fig.add_vline(x=obs_dep, line_color="red", line_width=5)
fig.update_layout(xaxis_title="Difference in Mean Encoded Causes",
                  yaxis_title="Frequency")
fig.show()


In [19]:
# Encode U.S._STATE as category codes
state_codes = df["U.S._STATE"].astype("category").cat.codes

# Drop rows where either value is NA
temp = pd.DataFrame({
    "missing": df["CA_MISSING"],
    "state": state_codes
}).dropna()

# Observed difference in means
obs_state = temp.groupby("missing")["state"].mean().loc[True] - \
            temp.groupby("missing")["state"].mean().loc[False]

obs_state


np.float64(0.5345335217550549)

In [20]:
N = 5000
null_stats_state = []

for _ in range(N):
    shuffled = np.random.permutation(temp["missing"])
    means_perm = temp.groupby(shuffled)["state"].mean()
    stat = means_perm.loc[True] - means_perm.loc[False]
    null_stats_state.append(stat)

null_stats_state = np.array(null_stats_state)

# P-value
p_state = np.mean(np.abs(null_stats_state) >= abs(obs_state))
p_state


np.float64(0.523)

In [21]:


# Convert null stats into a DataFrame for plotting
df_null_state = pd.DataFrame({"null_stat": null_stats_state})

fig = px.histogram(
    df_null_state,
    x="null_stat",
    nbins=50,
    title="Null Distribution of Test Statistic (Missingness vs U.S. State)"
)

# Add observed statistic line
fig.add_vline(
    x=obs_state,
    line_width=5,
    line_color="red",
    annotation_text="Observed Statistic=0.5345",
    annotation_position="top right"
)

fig.update_layout(
    xaxis_title="Difference in Means (Missing vs Not Missing)",
    yaxis_title="Frequency"
)

fig.show()


## Step 4: Hypothesis Testing

In [22]:
weather_mean = df[df["IS_SEVERE"]]["OUTAGE.DURATION.MIN"].mean()
nonweather_mean = df[~df["IS_SEVERE"]]["OUTAGE.DURATION.MIN"].mean()

obs_stat = weather_mean - nonweather_mean
obs_stat

np.float64(2537.8062533051298)

In [23]:
N = 5000
null_stats = []

for _ in range(N):
    shuffled = np.random.permutation(df["IS_SEVERE"].values)

    weather_mean_perm = df[shuffled]["OUTAGE.DURATION.MIN"].mean()
    nonweather_mean_perm = df[~shuffled]["OUTAGE.DURATION.MIN"].mean()

    stat = weather_mean_perm - nonweather_mean_perm
    null_stats.append(stat)

null_stats = np.array(null_stats)

# One-sided p-value
p_value = np.mean(null_stats >= obs_stat)
p_value

np.float64(0.0)

In [24]:
import plotly.express as px

fig = px.histogram(
    null_stats,
    nbins=50,
    title="Null Distribution of Difference in Means (Weather vs Non-Weather)"
)

fig.add_vline(x=obs_stat, line_width=5, line_color="red")
fig.update_layout(xaxis_title="Difference in Mean Outage Duration (min)",
                  yaxis_title="Frequency")
fig.show()


## Step 5: Framing a Prediction Problem

# Predicting Outage Duration ( > 24hrs )

For Step 5, We define the following prediction problem:

Goal:
Predict whether a power outage lasts more than 24 hours (1440 minutes) based on information available at the start of the outage.

## Type of prediction problem

Binary classification problem:

1 = outage duration > 1440 minutes (severe outage)

0 = outage duration ≤ 1440 minutes (non-severe outage)

### Response variable

I create a new variable:

In [25]:
df["LONG_OUTAGE"] = (df["OUTAGE.DURATION.MIN"] > 1440).astype(int)

# Features we will use
At the start of an outage, the utility already knows:

* CAUSE.CATEGORY (reported cause category)

* MONTH and YEAR

* CLIMATE.CATEGORY (climate episode that year)

* NERC.REGION

* ANOMALY.LEVEL

* RES.CUSTOMERS (baseline state-level infrastructure info)

# Evaluation metric: F1-score

Since the groups are more likely imbalanced as severe outages are less common, accuracy would not ideal. We will use **F1-score**, because it balances precision and recall

It penalizes false negatives (predicting an outage will be short when it becomes long), and it works better for imbalanced binary classification problems


## Step 6: Baseline Model

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [27]:
df["LONG_OUTAGE"] = (df["OUTAGE.DURATION.MIN"] > 1440).astype(int)

df["YEAR"] = df["YEAR"].astype(float)
df["ANOMALY.LEVEL"] = df["ANOMALY.LEVEL"].astype(float)

baseline_features = [
    "YEAR",
    "ANOMALY.LEVEL",
    "CLIMATE.CATEGORY",
    "NERC.REGION"
]

X = df[baseline_features]
y = df["LONG_OUTAGE"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

numeric_features = ["YEAR", "ANOMALY.LEVEL"]
categorical_features = ["CLIMATE.CATEGORY", "NERC.REGION"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="median"), numeric_features),
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ]
)

baseline_model = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("clf", LogisticRegression(max_iter=10000))
])

baseline_model.fit(X_train, y_train)
y_pred = baseline_model.predict(X_test)

baseline_f1 = f1_score(y_test, y_pred)
baseline_f1


0.4260869565217391

In [28]:
print("Baseline Model F1-score:", baseline_f1)

Baseline Model F1-score: 0.4260869565217391


## Step 7: Final Model

In [29]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np

In [30]:
# TARGET VARIABLE: LONG_OUTAGE

# Outages lasting > 1440 min (1 day)
df["LONG_OUTAGE"] = (df["OUTAGE.DURATION.MIN"] > 1440).astype(int)

In [31]:
# Engineered features
df["SALES_PER_CUSTOMER"] = df["TOTAL.SALES"] / df["TOTAL.CUSTOMERS"]
df["CUSTOMERS_PER_AREA"] = df["TOTAL.CUSTOMERS"] / df["AREAPCT_URBAN"]

df["SALES_PER_CUSTOMER"] = df["SALES_PER_CUSTOMER"].replace([np.inf, -np.inf], np.nan)
df["CUSTOMERS_PER_AREA"] = df["CUSTOMERS_PER_AREA"].replace([np.inf, -np.inf], np.nan)

features = [
    "YEAR", "MONTH", "ANOMALY.LEVEL",
    "NERC.REGION", "CLIMATE.REGION",

    # Economics / Infrastructure
    "TOTAL.PRICE", "TOTAL.SALES", "TOTAL.CUSTOMERS",
    "PC.REALGSP.STATE", "UTIL.REALGSP",

    # Urbanization
    "POPULATION", "POPPCT_URBAN", "POPDEN_URBAN", "POPDEN_RURAL", "AREAPCT_URBAN",

    # Engineered features
    "SALES_PER_CUSTOMER", "CUSTOMERS_PER_AREA"
]

y = df["LONG_OUTAGE"]
X = df[features]

cat_cols = ["NERC.REGION", "CLIMATE.REGION"]
num_cols = [col for col in features if col not in cat_cols]



In [32]:
# Drop rows with missing values in selected features
X = X.replace([np.inf, -np.inf], np.nan)
df_model = pd.concat([X, y], axis=1).dropna()
X = df_model[features]
y = df_model["LONG_OUTAGE"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Preprocessing

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ]
)

# Model 1 — Random Forest

rf_pipeline = Pipeline([
    ("prep", preprocess),
    ("clf", RandomForestClassifier(random_state=42))
])

rf_params = {
    "clf__n_estimators": [200, 400, 600],
    "clf__max_depth": [8, 12, 16, None],
    "clf__min_samples_split": [2, 5, 10]
}

rf_search = GridSearchCV(
    rf_pipeline, rf_params, scoring="f1", cv=4, n_jobs=-1, verbose=1
)
rf_search.fit(X_train, y_train)

rf_pred = rf_search.predict(X_test)
rf_f1 = f1_score(y_test, rf_pred)

print("Random Forest F1:", rf_f1)
print("Best RF Params:", rf_search.best_params_)

Fitting 4 folds for each of 36 candidates, totalling 144 fits
Random Forest F1: 0.61003861003861
Best RF Params: {'clf__max_depth': 16, 'clf__min_samples_split': 10, 'clf__n_estimators': 200}


In [33]:
rf_pipeline = Pipeline([
    ("prep", preprocess),
    ("clf", RandomForestClassifier(
        random_state=42,
        class_weight="balanced"
    ))
])

rf_params = {
    "clf__n_estimators": [300, 500],
    "clf__max_depth": [12, 20, None],
    "clf__min_samples_split": [2, 5]
}

rf_search = GridSearchCV(
    rf_pipeline,
    rf_params,
    scoring="f1",
    cv=4,
    n_jobs=-1,
    verbose=1
)

rf_search.fit(X_train, y_train)

y_pred = rf_search.predict(X_test)
rf_f1 = f1_score(y_test, y_pred)

print("Improved RF F1:", rf_f1)
print("Best Params:", rf_search.best_params_)


Fitting 4 folds for each of 12 candidates, totalling 48 fits
Improved RF F1: 0.624113475177305
Best Params: {'clf__max_depth': 12, 'clf__min_samples_split': 2, 'clf__n_estimators': 500}


In [34]:
from sklearn.tree import DecisionTreeClassifier

dt_pipeline = Pipeline([
    ("prep", preprocess),
    ("clf", DecisionTreeClassifier(random_state=42, class_weight="balanced"))
])

dt_params = {
    "clf__max_depth": [6, 10, 14, 18, None],
    "clf__min_samples_split": [2, 5, 10],
    "clf__min_samples_leaf": [1, 2, 4]
}

dt_search = GridSearchCV(
    dt_pipeline,
    dt_params,
    scoring="f1",
    cv=4,
    verbose=1,
    n_jobs=-1
)

dt_search.fit(X_train, y_train)
dt_pred = dt_search.predict(X_test)
dt_f1 = f1_score(y_test, dt_pred)

print("Decision Tree F1:", dt_f1)
print("Best Tree Params:", dt_search.best_params_)


Fitting 4 folds for each of 45 candidates, totalling 180 fits
Decision Tree F1: 0.6622950819672131
Best Tree Params: {'clf__max_depth': 14, 'clf__min_samples_leaf': 4, 'clf__min_samples_split': 10}


## Step 8: Fairness Analysis

In [35]:
# TODO